import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate0=900
rateNet=480         #236                                                           # Real rate , Load data+ '0'
cls_num=2
for i in range (1,cls_num+1):                                                  #Cls from 0to1 >>   replace:(1,cls_num+1)
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=WormsTwoClass0(i)                # Real rate    >>   Load data+'0'

os.chdir(cwd)
clses_lens=np.array([])
ecg=np.array(ecg1)                                                             #   4 case 1 to 0
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):                                                    #Cls from 0to1 >> (1,cls_num) 
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
btch=int(min(clses_lens))

first_train_shape= (77, 901)
first_test_shape= (181, 901)
classes_quantity= 2
tr_lbls=	 {1, 2}
Count_labels= [33 44]
max(train_feature_Altitude)= 4.1961
min(train_feature_Altitude)= -4.8873
first_train_sample=
 [ 1.        -0.77859   -0.74444   -0.72546   -0.70269   -0.71408
 -0.72167   -0.73305   -0.71787   -0.73685   -0.76341   -0.77479
 -0.76341   -0.74823   -0.72546   -0.73305   -0.73305   -0.74064
 -0.74444   -0.76341   -0.7672    -0.76341   -0.75582   -0.74823
 -0.74823   -0.75582   -0.74823   -0.75962   -0.75203   -0.74823
 -0.74823   -0.73305   -0.73305   -0.74064   -0.771     -0.80895
 -0.83931   -0.86587   -0.87725   -0.87346   -0.919     -0.86207
 -0.86207   -0.85069   -0.82792   -0.85069   -0.78238   -0.7672
 -0.74444   -0.72546   -0.75203   -0.6951    -0.6951    -0.72167
 -0.72926   -0.74444   -0.74444   -0.79756   -0.75203   -0.67233
 -0.64577   -0.67613   -0.65716   -0.61541   -0.63818   -0.63439
 -0.59264   -0.61162   -0.5547    -0.42567   -0.30044   -0.21316
 -0.22455 

In [3]:
ecg2[20]

array([ 4.8322e-01,  4.2126e-01,  3.4951e-01,  2.7451e-01,  2.4842e-01,
        2.0277e-01,  1.5059e-01,  1.3428e-01,  1.2776e-01,  9.5150e-02,
        1.0819e-01,  7.5584e-02,  8.2106e-02,  7.2323e-02,  7.2323e-02,
        3.6451e-02, -6.4641e-02, -7.1163e-02, -1.5921e-01, -1.2986e-01,
       -1.2334e-01, -9.7252e-02, -9.3991e-02, -4.8336e-02,  4.6234e-02,
        6.9062e-02,  1.4081e-01,  2.0929e-01,  2.8103e-01,  3.5277e-01,
        4.1147e-01,  5.3865e-01,  5.6148e-01,  5.1909e-01,  4.9300e-01,
        5.2887e-01,  5.5170e-01,  5.4518e-01,  5.2561e-01,  4.8974e-01,
        4.4734e-01,  3.8212e-01,  2.5820e-01,  1.9298e-01,  1.2450e-01,
        8.5367e-02,  6.2540e-02,  1.3624e-02, -1.5726e-02, -2.5509e-02,
        5.7974e-04,  3.8408e-03,  5.9279e-02,  9.5150e-02,  1.5059e-01,
        2.2885e-01,  3.4951e-01,  3.8212e-01,  4.6365e-01,  4.6365e-01,
        6.0714e-01,  6.5605e-01,  6.8866e-01,  6.9518e-01,  7.0497e-01,
        7.0823e-01,  6.4627e-01,  6.0387e-01,  5.9735e-01,  5.68

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [4]:
smpl_rte=rate0                                                   # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.empty((0,smpl_rte+1))

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    windws=np.append(windws,vars()['wndws'+str(cls)],axis=0)
    
print("each class and its windows =")

furir_windws0=np.fft.fft(np.array(windws[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([windws[:,-1]])),axis=1)
print("shape furir_windws=", np.shape(furir_windws))

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
shape furir_windws0= (77, 900)
shape furir_windws= (77, 901)
1 33	2 44	
 max = 44


In [5]:
#clses=3                                   #تعیین میزان افزایش نمونه (تولید داده)
mx_wndws=int(1.5*mx_wndws)  #mx_wndws*0.3  # =classes-1 برای بررسی الگوریتم تعداد کمی کلاس آزمایش شد
print(mx_wndws)

66


<h1><center><div style="direction:rtl;font-family:B Nazanin">Augment Random CNN</div></center></h1>

In [6]:
rpt_cnn_clsfctn=3
rslt=np.array([])
rslts=np.array([])
xtrain=np.empty((0,len(wndws1[0])), float) 


for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug10'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug11'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug12'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug20'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug21'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug22'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug30'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug31'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug32'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug40'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug41'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug42'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug50'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug51'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug52'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (1,cls_num+1):                                     #تعریف آرایه ی پنجره های کلاس ها
    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
    #print("vlum_win=",vlum_win)
    vars()['ecg_aug10'+str(i)]=np.append(vars()['ecg_aug10'+str(i)],scl_rndm_beat(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug11'+str(i)]=np.append(vars()['ecg_aug11'+str(i)],scl_rndm_beat(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug12'+str(i)]=np.append(vars()['ecg_aug12'+str(i)],scl_rndm_beat(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug20'+str(i)]=np.append(vars()['ecg_aug20'+str(i)],MgWrp_rndm_beat(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug21'+str(i)]=np.append(vars()['ecg_aug21'+str(i)],MgWrp_rndm_beat(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug22'+str(i)]=np.append(vars()['ecg_aug22'+str(i)],MgWrp_rndm_beat(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug30'+str(i)]=np.append(vars()['ecg_aug30'+str(i)],TimWrp_rndm_beat(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug31'+str(i)]=np.append(vars()['ecg_aug31'+str(i)],TimWrp_rndm_beat(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug32'+str(i)]=np.append(vars()['ecg_aug32'+str(i)],TimWrp_rndm_beat(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug40'+str(i)]=np.append(vars()['ecg_aug40'+str(i)],GDO_rndm_beat(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug41'+str(i)]=np.append(vars()['ecg_aug41'+str(i)],GDO_rndm_beat(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug42'+str(i)]=np.append(vars()['ecg_aug42'+str(i)],GDO_rndm_beat(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug50'+str(i)]=np.append(vars()['ecg_aug50'+str(i)],frqnc_rndm_beat(furir_windws,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug51'+str(i)]=np.append(vars()['ecg_aug51'+str(i)],frqnc_rndm_beat(furir_windws,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug52'+str(i)]=np.append(vars()['ecg_aug52'+str(i)],frqnc_rndm_beat(furir_windws,i,vlum_win,mag=.2),axis=0)    

for methd in range(1,6):
    for Mgntud in range(0,3):
        xtrain=np.empty((0,len(wndws1[0])), float) 
        rprt=np.empty((0,3),float)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            for i in range (1,cls_num+1):   
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                xtrain=np.append(xtrain,vars()['ecg_aug'+str(methd)+str(Mgntud)+str(i)],axis=0)
                #print('Up to class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (1,cls_num+1):                
                cls=i
                #print('\n cls', i, ' >> ')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                #print('after normalizing >>')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
            #print("each class and its windows =")                    #تجمیع کلاس های تست
            cls_wndws=np.array([])
            for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                cls_wndws=np.append(cls_wndws,wns)
                #print(cls, wns, end='\t')
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
            #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
            #print('\n max instance in classes of test windows =', mx_wndws)

            #print('min tst =', np.min(wndws_test[:,:-1]))
            #print('max tst =', np.max(wndws_test[:,:-1]))

            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            #fullprint(ytest)

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(signal.resample(Xtrain,rateNet,axis=1))
            y_train=np.array(ytrain)
            X_test=np.array(signal.resample(Xtest,rateNet,axis=1))
            y_test=np.array(ytest)
            X_valid=np.array(signal.resample(Xtrain,rateNet,axis=1))
            y_valid=np.array(ytrain)
            
            if min(y_train)==1:
                y_train=y_train-1
                y_test=y_test-1
                y_valid =y_valid-1

            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            #VGG16_1D

            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :

            verbose, epochs, batch_size = 0, 1000, btch
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data

            model_crs = Sequential()
            BatchNormalization()
            model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
            #model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
            #model_crs.add(Dropout(0.5))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            '''
            model_crs.add(Flatten())
            #model_crs.add(Dropout(0.5))
            model_crs.add(Dense(2000, activation='relu'))
            model_crs.add(Dense(n_outputs, activation='softmax'))
            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
            # evaluate model_crs
            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)


            
            predict_x=model_crs.predict(X_test)              # Function 1
            y_pred_crs=np.argmax(predict_x,axis=1)           # function from augment.py to remove 0 index predictions
            
            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,np.array([rprt_row]),axis=0)
        maxm=rprt[np.argmax(rprt[:,1])]
        avrg=np.mean(rprt,axis=0)
        rslts=np.append(rslts,['mthd '+str(methd)+'  Mgntud '+str(Mgntud)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt[:,1])))],axis=0)

C:\Users\user\Thesis\Classifying\CNNsplit\CNN_Art_WormsTwoClass77_71\augment.py:658: ComplexWarning: Casting complex values to real discards the imaginary part
  p_wn = np.float16(np.concatenate((P,pl), axis=1))


6/6 [==============================] - 0s 2ms/step


In [7]:
vlum_win

22

In [8]:
rslts

array(['mthd 1  Mgntud 0  average= [66 64 65]  max= [68 67 68]  var= 9',
       'mthd 1  Mgntud 1  average= [68 65 67]  max= [70 68 70]  var= 5',
       'mthd 1  Mgntud 2  average= [66 64 65]  max= [67 65 66]  var= 1',
       'mthd 2  Mgntud 0  average= [67 65 66]  max= [69 67 68]  var= 1',
       'mthd 2  Mgntud 1  average= [67 64 65]  max= [70 67 69]  var= 5',
       'mthd 2  Mgntud 2  average= [65 63 65]  max= [67 66 67]  var= 3',
       'mthd 3  Mgntud 0  average= [65 65 65]  max= [71 70 71]  var= 16',
       'mthd 3  Mgntud 1  average= [66 64 66]  max= [67 66 67]  var= 2',
       'mthd 3  Mgntud 2  average= [63 62 63]  max= [67 66 67]  var= 6',
       'mthd 4  Mgntud 0  average= [65 63 64]  max= [69 67 68]  var= 13',
       'mthd 4  Mgntud 1  average= [65 63 64]  max= [68 67 68]  var= 9',
       'mthd 4  Mgntud 2  average= [66 65 66]  max= [67 66 67]  var= 1',
       'mthd 5  Mgntud 0  average= [66 64 65]  max= [67 65 67]  var= 3',
       'mthd 5  Mgntud 1  average= [66 64 65]  ma

<h1><center><div style="direction:rtl;font-family:B Nazanin">Augment Marginal CNN</div></center></h1>

In [9]:
rpt_cnn_clsfctn=3
rslt=np.array([])
rslts=np.array([])
xtrain=np.empty((0,len(wndws1[0])), float) 
rprt=np.empty((0,3),float)


for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug10'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug11'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug12'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug20'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug21'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug22'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug30'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug31'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug32'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug40'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug41'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug42'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug50'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug51'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug52'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (1,cls_num+1):                                     #تعریف آرایه ی پنجره های کلاس ها
    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
    #print("vlum_win=",vlum_win)
    vars()['ecg_aug10'+str(i)]=np.append(vars()['ecg_aug10'+str(i)],sclRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug11'+str(i)]=np.append(vars()['ecg_aug11'+str(i)],sclRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug12'+str(i)]=np.append(vars()['ecg_aug12'+str(i)],sclRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug20'+str(i)]=np.append(vars()['ecg_aug20'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug21'+str(i)]=np.append(vars()['ecg_aug21'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug22'+str(i)]=np.append(vars()['ecg_aug22'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug30'+str(i)]=np.append(vars()['ecg_aug30'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug31'+str(i)]=np.append(vars()['ecg_aug31'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug32'+str(i)]=np.append(vars()['ecg_aug32'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug40'+str(i)]=np.append(vars()['ecg_aug40'+str(i)],GDORaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug41'+str(i)]=np.append(vars()['ecg_aug41'+str(i)],GDORaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug42'+str(i)]=np.append(vars()['ecg_aug42'+str(i)],GDORaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug50'+str(i)]=np.append(vars()['ecg_aug50'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug51'+str(i)]=np.append(vars()['ecg_aug51'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug52'+str(i)]=np.append(vars()['ecg_aug52'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.2),axis=0)    

for methd in range(1,6):
    for Mgntud in range(0,3):
        xtrain=np.empty((0,len(wndws1[0])), float) 
        rprt=np.empty((0,3),float)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            for i in range (1,cls_num+1):   
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                xtrain=np.append(xtrain,vars()['ecg_aug'+str(methd)+str(Mgntud)+str(i)],axis=0)
                #print('Up to class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (1,cls_num+1):                
                cls=i
                #print('\n cls', i, ' >> ')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                #print('after normalizing >>')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
            #print("each class and its windows =")                    #تجمیع کلاس های تست
            cls_wndws=np.array([])
            for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                cls_wndws=np.append(cls_wndws,wns)
                #print(cls, wns, end='\t')
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
            #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
            #print('\n max instance in classes of test windows =', mx_wndws)

            #print('min tst =', np.min(wndws_test[:,:-1]))
            #print('max tst =', np.max(wndws_test[:,:-1]))

            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            #fullprint(ytest)

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(signal.resample(Xtrain,rateNet,axis=1))
            y_train=np.array(ytrain)
            X_test=np.array(signal.resample(Xtest,rateNet,axis=1))
            y_test=np.array(ytest)
            X_valid=np.array(signal.resample(Xtrain,rateNet,axis=1))
            y_valid=np.array(ytrain)

            if min(y_train)==1:
                y_train=y_train-1
                y_test=y_test-1
                y_valid =y_valid-1
    
            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            ####### NETWORK #######
            #!=Yildirin

            #=VGG16_1D

            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :

            verbose, epochs, batch_size = 0, 1000, btch
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data

            model_crs = Sequential()
            BatchNormalization()
            model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
            #model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
            #model_crs.add(Dropout(0.5))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            '''
            model_crs.add(Flatten())
            #model_crs.add(Dropout(0.5))
            model_crs.add(Dense(2000, activation='relu'))
            model_crs.add(Dense(n_outputs, activation='softmax'))
            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
            # evaluate model_crs
            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)

            predict_x=model_crs.predict(X_test)                # Function 1
            y_pred_crs=np.argmax(predict_x,axis=1)             # function from augment.py to remove 0 index predictions

            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,np.array([rprt_row]),axis=0)
        maxm=rprt[np.argmax(rprt[:,1])]
        avrg=np.mean(rprt,axis=0)
        rslts=np.append(rslts,['mthd '+str(methd)+'  Mgntud '+str(Mgntud)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt[:,1])))],axis=0)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\user\Thesis\Classifying\CNNsplit\CNN_Art_WormsTwoClass77_71\augment.py:1504: ComplexWarning: Casting complex values to real discards the imaginary part
  return(np.float16(p_aug_raw))


6/6 [==============================] - 0s 3ms/step


In [10]:
rslts

array(['mthd 1  Mgntud 0  average= [69 67 68]  max= [71 69 70]  var= 4',
       'mthd 1  Mgntud 1  average= [65 62 63]  max= [66 63 65]  var= 1',
       'mthd 1  Mgntud 2  average= [67 65 67]  max= [68 66 67]  var= 0',
       'mthd 2  Mgntud 0  average= [67 65 66]  max= [69 66 67]  var= 0',
       'mthd 2  Mgntud 1  average= [69 67 68]  max= [72 70 71]  var= 4',
       'mthd 2  Mgntud 2  average= [69 68 69]  max= [72 71 72]  var= 5',
       'mthd 3  Mgntud 0  average= [64 61 63]  max= [66 63 65]  var= 4',
       'mthd 3  Mgntud 1  average= [67 64 66]  max= [70 68 69]  var= 8',
       'mthd 3  Mgntud 2  average= [67 66 67]  max= [70 70 70]  var= 7',
       'mthd 4  Mgntud 0  average= [67 64 65]  max= [69 65 67]  var= 1',
       'mthd 4  Mgntud 1  average= [68 67 68]  max= [71 70 71]  var= 5',
       'mthd 4  Mgntud 2  average= [63 61 62]  max= [63 62 63]  var= 1',
       'mthd 5  Mgntud 0  average= [67 65 67]  max= [70 67 69]  var= 2',
       'mthd 5  Mgntud 1  average= [67 65 66]  max=